In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Download and Installations

In [3]:
path_to_drive = "drive/MyDrive/TSD/"

In [ ]:
import nltk
nltk.download('stopwords')
!pip3 install transformers
!pip install keras-self-attention

# 2. Loading Dataset

In [8]:
import warnings
warnings.filterwarnings("ignore")
import cross_validator as cross_validator
import datahandler as datahandler
from datamodel import DataModel
import outputmaker as outputmaker
import metrics as metrics
from dl_utils import *
from tqdm import tqdm
import utils

train = datahandler.load_train(path_to_drive+'tsd_train.csv', verbose=True)
dev = datahandler.load_train(path_to_drive+'tsd_trial.csv', verbose=True)
test = datahandler.load_train(path_to_drive+'tsd_test.csv', verbose=True)

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."


,spans,text
0,"[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",Because he's a moron and a bigot. It's not any...
1,"[29, 30, 31, 32, 33, 34]",How about we stop protecting idiots and let na...
2,"[166, 167, 168, 169, 170, 171]","If people were smart, they would Boycott th..."


,text,spans
0,That's right. They are not normal. And I am st...,"[84, 85, 86, 87, 88, 89, 90, 91, 133, 134, 135..."
1,"""Watch people die from taking away their healt...","[81, 82, 83, 84, 85, 86]"
2,tens years ago i contacted the PDR and suggest...,[]


In [9]:
datamodel = DataModel(model='crf', remove_stop_words=False)

output_maker = outputmaker.crf_output

evaluator = metrics

In [10]:
X_train, y_train, train_taboo_words, train_texts = cross_validator.load_data(train, 
                                                    [i for i in range(train.shape[0])],
                                                    datamodel, logger=True)

X_dev, y_dev, dev_taboo_words, dev_texts = cross_validator.load_data(dev, 
                                                    [i for i in range(dev.shape[0])], 
                                                    datamodel, logger=True)

X_test, y_test, _, test_texts = cross_validator.load_data(test, 
                                                    [i for i in range(test.shape[0])], 
                                                    datamodel, logger=True)

100%|██████████| 2000/2000 [01:14<00:00, 26.92it/s]


In [11]:
X_dev[0]

[('Because', 'IN', 'normal', [0, 1, 2, 3, 4, 5, 6]),
 ('he', 'PRP', 'normal', [8, 9]),
 ('a', 'DT', 'normal', [13]),
 ('moron', 'NN', 'toxic', [15, 16, 17, 18, 19]),
 ('and', 'CC', 'normal', [21, 22, 23]),
 ('a', 'DT', 'normal', [25]),
 ('bigot', 'NNP', 'toxic', [27, 28, 29, 30, 31]),
 ('It', 'PRP', 'normal', [34, 35]),
 ('not', 'RB', 'normal', [39, 40, 41]),
 ('any', 'DT', 'normal', [43, 44, 45]),
 ('more', 'RBR', 'normal', [47, 48, 49, 50]),
 ('complicated', 'JJ', 'normal', [52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]),
 ('than', 'IN', 'normal', [64, 65, 66, 67]),
 ('that', 'DT', 'normal', [69, 70, 71, 72])]

# 3. Preprations for modeling

In [12]:
X_train = preprocessing(X_train, clean=False)
X_dev = preprocessing(X_dev, clean=False)
X_test = preprocessing(X_test, clean=False)

100%|██████████| 2000/2000 [00:00<00:00, 56576.95it/s]


In [13]:
words_list = X_train + X_dev + X_test
word2idx, idx2word, tag2idx, idx2tag, max_len, max_features, n_tags = get_indecies(words_list)

n_tags:2, n_words:21788

max_len:215 
word2idx['fuck']:1140,  tag2idx['normal']:2

Maximum features:21790


In [14]:
X_train_seq, y_train_seq = padsequences(X_train, word2idx, tag2idx, max_len)

X_dev_seq, y_dev_seq = padsequences(X_dev, word2idx, tag2idx, max_len)

X_test_seq, y_test_seq = padsequences(X_test, word2idx, tag2idx, max_len)

100%|██████████| 2000/2000 [00:00<00:00, 108443.00it/s]


# 4.Representations

In [15]:
glove = load_glove(path_to_drive+'glove.840B.300d.txt')
gpt2 = load_pkl(path_to_drive+"__vectors__gpt2__.sav")
roberta= load_pkl(path_to_drive+"__vectors__roberta__.sav")

2196017it [02:50, 12842.65it/s]


In [16]:
glove_embed_size = 300
glove_embeddings = get_glove_features(word2idx, glove, embed_size=glove_embed_size)

transformer_embed_size = 768
gpt2_embeddings = get_gpt2_features(word2idx, gpt2, embed_size=transformer_embed_size)
roberta_embeddings = get_roberta_features(word2idx, roberta, embed_size=transformer_embed_size)

rotberta_gpt2_embed_size = 768
rotberta_gpt2_embeddings = get_roberta_gpt2_features(word2idx, roberta, gpt2, 
                                                    embed_size=rotberta_gpt2_embed_size)

glove_transformer_embed_size = 1068
glove_gpt2_embeddings = get_glove_gpt2_features(word2idx, glove, gpt2,
                                                embed_size=glove_transformer_embed_size)
glove_roberta_embeddings = get_glove_roberta_features(word2idx, glove, roberta,
                                                embed_size=glove_transformer_embed_size)

ensemble_embed_size = 1068
ensemble_embeddings = ensemble_features(word2idx, glove, gpt2, roberta, 
                                        embed_size=ensemble_embed_size)

  0%|          | 0/21790 [00:00<?, ?it/s]

GLOVE:::OOV:6330/21790 


100%|██████████| 21790/21790 [00:00<00:00, 29310.48it/s]


# 5. Modelings

In [14]:
def get_models(name, embedding, embed_size, layers = ["lstm", "gru"], random_embedding=False):

    models = [ ] 
    for i in range(1, 3):
        for layer in layers:
            models.append(
            {"model":get_model(max_len, max_features, embed_size, embedding, n_tags, 
                                number=i, layer=layer, attention=False, random_embedding=random_embedding),
            "name":name+"_bi"+layer+"_"+str(i)+"_layer"})
            print("Initialize: ", name+"_bi"+layer+"_"+str(i)+"_layer")

            models.append(
            {"model":get_model(max_len, max_features, embed_size, embedding, n_tags, 
                                number=i, layer=layer, attention=True, random_embedding=random_embedding),
            "name":name+"_bi"+layer+"_"+str(i)+"_layer_attention"})
            print("Initialize: ", name+"_bi"+layer+"_"+str(i)+"_layer_attention")
    return models

glove_models = get_models("glove", glove_embeddings, glove_embed_size) 
gpt2_models = get_models("gpt2", gpt2_embeddings, transformer_embed_size)
roberta_models = get_models("roberta", roberta_embeddings, transformer_embed_size)
random_models = get_models("random", None, transformer_embed_size, random_embedding=True)
roberta_gpt2_models = get_models("roberta_gpt2", rotberta_gpt2_embeddings, rotberta_gpt2_embed_size)
glove_gp2_models = get_models("glove_gpt2", glove_gpt2_embeddings, glove_transformer_embed_size)
glove_roberta_models = get_models("glove_roberta", glove_roberta_embeddings, glove_transformer_embed_size)
ensemble_models = get_models("ensemble", ensemble_embeddings, ensemble_embed_size)

models = glove_models + gpt2_models + roberta_models + random_models 
models += roberta_gpt2_models + glove_gp2_models + glove_roberta_models + ensemble_models

print("Number of intitialized models:", len(models))



Initialize:  glove_bilstm_1_layer
Initialize:  glove_bilstm_1_layer_attention
Initialize:  glove_bigru_1_layer
Initialize:  glove_bigru_1_layer_attention
Initialize:  glove_bilstm_2_layer
Initialize:  glove_bilstm_2_layer_attention
Initialize:  glove_bigru_2_layer
Initialize:  glove_bigru_2_layer_attention
Initialize:  gpt2_bilstm_1_layer
Initialize:  gpt2_bilstm_1_layer_attention
Initialize:  gpt2_bigru_1_layer
Initialize:  gpt2_bigru_1_layer_attention
Initialize:  gpt2_bilstm_2_layer
Initialize:  gpt2_bilstm_2_layer_attention
Initialize:  gpt2_bigru_2_layer
Initialize:  gpt2_bigru_2_layer_attention
Initialize:  roberta_bilstm_1_layer
Initialize:  roberta_bilstm_1_layer_attention
Initialize:  roberta_bigru_1_layer
Initialize:  roberta_bigru_1_layer_attention
Initialize:  roberta_bilstm_2_layer
Initialize:  roberta_bilstm_2_layer_attention
Initialize:  roberta_bigru_2_layer
Initialize:  roberta_bigru_2_layer_attention
Initialize:  random_bilstm_1_layer
Initialize:  random_bilstm_1_la

In [ ]:
results = []
for model in models:
    print("Working on Model:", model['name'])
    history = model['model'].fit(X_train_seq, y_train_seq, batch_size=32, epochs=2, 
                                 validation_data=(X_dev_seq, y_dev_seq), verbose=0)
    
    y_dev_predict, y_dev_proba = get_prediction_spans(model['model'].predict(X_dev_seq), 
                                                      X_dev_seq, idx2tag)
    y_dev_predict_span = output_maker(X_dev, dev_texts, y_dev_predict)

    y_test_predict, y_test_proba = get_prediction_spans(model['model'].predict(X_test_seq), 
                                                      X_test_seq, idx2tag)
    y_test_predict_span = output_maker(X_test, test_texts, y_test_predict)
    
    f1, p, r = evaluator.evaluate(gold = y_dev, predictions = y_dev_predict_span)
    model['dev-f1'], model['dev-p'], model['dev-r'] = f1, p, r
    print("\nDEV: F1:{}, \t P:{}, \t R:{}".format(f1, p, r))

    f1, p, r = evaluator.evaluate(gold = y_test, predictions = y_test_predict_span)
    model['test-f1'], model['test-p'], model['test-r'] = f1, p, r
    print("\nTEST: F1:{}, \t P:{}, \t R:{}".format(f1, p, r))
    print("---------------------------------------------------------------")
    
    model['history'] = history
    model['config'] = model['model'].to_json()
    
    dev[model['name']] = y_dev_predict_span
    test[model['name']] = y_test_predict_span
    model.pop('model')
    result.append(model)
    break

Working on Model: glove_bilstm_1_layer


# 5. Modeling V2

In [29]:
def train_model(name, embedding, embed_size, X_train_seq, y_train_seq,
                X_dev_seq, y_dev_seq, X_test_seq, y_test_seq, 
                dev, X_dev, y_dev, dev_texts, test, X_test, y_test, test_texts, idx2tag,
                layers = ["lstm", "gru"], random_embedding=False, BATCH=32, EPOCH=10):
    models = [ ] 
    for i in range(1, 3):
        for layer in layers:
            for attention in [False, True]:
                if attention:
                    run_name = name+"_bi"+layer+"_"+str(i)+"_layer_attention"
                else:
                    run_name = name+"_bi"+layer+"_"+str(i)+"_layer"
                print("Working on: ", run_name)

                model = get_model(max_len, max_features, embed_size, embedding, 
                                  n_tags, number=i, layer=layer, attention=attention, 
                                  random_embedding=random_embedding)

                history = model.fit(X_train_seq, y_train_seq, batch_size=BATCH, epochs=EPOCH, 
                                    validation_data=(X_dev_seq, y_dev_seq), verbose=0)

                y_dev_predict, _ = get_prediction_spans(model.predict(X_dev_seq), 
                                                        X_dev_seq, idx2tag)
                y_dev_predict_span = output_maker(X_dev, dev_texts, y_dev_predict)

                y_test_predict, _ = get_prediction_spans(model.predict(X_test_seq), 
                                                         X_test_seq, idx2tag)
                y_test_predict_span = output_maker(X_test, test_texts, y_test_predict)

                dev_f1, dev_p, dev_r = evaluator.evaluate(gold = y_dev, 
                                                          predictions = y_dev_predict_span)
                print("\nDEV: F1:{}, \t P:{}, \t R:{}".format(dev_f1, dev_p, dev_r))
                test_f1, test_p, test_r = evaluator.evaluate(gold = y_test, 
                                                             predictions = y_test_predict_span)
                print("\nTEST: F1:{}, \t P:{}, \t R:{}".format(test_f1, test_p, test_r))
                model_info = {
                    "name":run_name,
                    #"history":history,
                    "config":model.to_json(),
                    "dev-result":{ "F1":dev_f1, "P":dev_p, "R":dev_r },
                    "test-result":{ "F1":test_f1, "P":test_p, "R":test_r },
                }
                dev[run_name] = y_dev_predict_span
                test[run_name] = y_test_predict_span
                print("---------------------------------------------------------------")
                models.append(model_info)
                del model
    return models, dev, test

In [25]:
#GLOVE
glove_models, dev, test = train_model("glove", glove_embeddings, glove_embed_size, 
                        X_train_seq, y_train_seq, X_dev_seq, y_dev_seq, X_test_seq, y_test_seq, 
                        dev, X_dev, y_dev, dev_texts, test, X_test, y_test, test_texts, 
                        idx2tag, random_embedding=False)

Working on:  glove_bilstm_1_layer




100%|██████████| 2000/2000 [00:00<00:00, 2840.72it/s]



DEV: F1:0.5933439452594426, 	 P:0.692727399323684, 	 R:0.599064276158495

TEST: F1:0.6345711717904563, 	 P:0.6430424909657134, 	 R:0.6818846342593257
---------------------------------------------------------------
Working on:  glove_bilstm_1_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2864.91it/s]



DEV: F1:0.5814528228364736, 	 P:0.6882275452070472, 	 R:0.574615251359855

TEST: F1:0.6315053308063742, 	 P:0.647073983195685, 	 R:0.66464784296597
---------------------------------------------------------------
Working on:  glove_bigru_1_layer


...


DEV: F1:0.581900541419565, 	 P:0.7024731120847837, 	 R:0.5614464748552733

TEST: F1:0.6535874568591744, 	 P:0.6769076345886258, 	 R:0.6717023817600346
---------------------------------------------------------------
Working on:  glove_bigru_2_layer


100%|██████████| 2000/2000 [00:00<00:00, 2927.69it/s]



DEV: F1:0.5979811650352987, 	 P:0.6976640874769653, 	 R:0.5957042794317434

TEST: F1:0.6335300780580434, 	 P:0.6453141167171984, 	 R:0.6730318660135038
---------------------------------------------------------------
Working on:  glove_bigru_2_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2852.26it/s]


DEV: F1:0.5722379793511155, 	 P:0.6819277253349778, 	 R:0.5613819271973783

TEST: F1:0.6392336157626282, 	 P:0.6558267008938164, 	 R:0.6645190897533937
---------------------------------------------------------------


In [32]:
dev.to_csv(path_to_drive+"trial.csv", index=False)

test.to_csv(path_to_drive+"test.csv", index=False)

utils.save_json(path_to_drive+"tsd_experiments_logs_glove_models.json", glove_models)

In [33]:
#GPT2
gpt2_models, dev, test = train_model("gpt2", gpt2_embeddings, transformer_embed_size, 
                        X_train_seq, y_train_seq, X_dev_seq, y_dev_seq, X_test_seq, y_test_seq, 
                        dev, X_dev, y_dev, dev_texts, test, X_test, y_test, test_texts, 
                        idx2tag, random_embedding=False)

Working on:  gpt2_bilstm_1_layer


100%|██████████| 2000/2000 [00:00<00:00, 2873.18it/s]



DEV: F1:0.6129786978911472, 	 P:0.634555770360232, 	 R:0.7232993595135786

TEST: F1:0.5684156742162274, 	 P:0.5166155849309709, 	 R:0.7335360979978383
---------------------------------------------------------------
Working on:  gpt2_bilstm_1_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2912.18it/s]



DEV: F1:0.610706796827132, 	 P:0.7018870328766567, 	 R:0.6193708758301738

TEST: F1:0.6384119111811389, 	 P:0.6368887796272124, 	 R:0.6950900792330174
---------------------------------------------------------------
Working on:  gpt2_bigru_1_layer


...


DEV: F1:0.6152352788272297, 	 P:0.726234702261056, 	 R:0.5922353760901371

TEST: F1:0.6763276144842764, 	 P:0.6947887938407482, 	 R:0.6964228823092393
---------------------------------------------------------------
Working on:  gpt2_bigru_2_layer


100%|██████████| 2000/2000 [00:00<00:00, 3051.39it/s]



DEV: F1:0.6417727088744084, 	 P:0.7410395424513354, 	 R:0.6522435544014941

TEST: F1:0.6235543027110854, 	 P:0.6275418019742806, 	 R:0.6793689533532437
---------------------------------------------------------------
Working on:  gpt2_bigru_2_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2969.30it/s]


DEV: F1:0.6493665723557466, 	 P:0.7722677333260862, 	 R:0.6410417647652913

TEST: F1:0.6596750254661106, 	 P:0.6650714605074182, 	 R:0.70350163240807
---------------------------------------------------------------


In [34]:
dev.to_csv(path_to_drive+"trial.csv", index=False)

test.to_csv(path_to_drive+"test.csv", index=False)

utils.save_json(path_to_drive+"tsd_experiments_logs_gpt2_models.json", gpt2_models)

In [36]:
#roberta
roberta_models, dev, test = train_model("roberta", roberta_embeddings, transformer_embed_size, 
                        X_train_seq, y_train_seq, X_dev_seq, y_dev_seq, X_test_seq, y_test_seq, 
                        dev, X_dev, y_dev, dev_texts, test, X_test, y_test, test_texts, 
                        idx2tag, random_embedding=False)

Working on:  roberta_bilstm_1_layer


100%|██████████| 2000/2000 [00:00<00:00, 2765.88it/s]



DEV: F1:0.5643525813518603, 	 P:0.6709752767361463, 	 R:0.5342818558322118

TEST: F1:0.6530989181969675, 	 P:0.6850516763765989, 	 R:0.6502567062283988
---------------------------------------------------------------
Working on:  roberta_bilstm_1_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2863.66it/s]



DEV: F1:0.5989069245850228, 	 P:0.6977010360378916, 	 R:0.5791791422706808

TEST: F1:0.6635659869772619, 	 P:0.6816547318078886, 	 R:0.6822285517930211
---------------------------------------------------------------
Working on:  roberta_bigru_1_layer


...


DEV: F1:0.620858543612296, 	 P:0.733420765227495, 	 R:0.6032213392760811

TEST: F1:0.657848464380668, 	 P:0.6710835881424306, 	 R:0.6875699258439912
---------------------------------------------------------------
Working on:  roberta_bigru_2_layer


100%|██████████| 2000/2000 [00:00<00:00, 3010.57it/s]



DEV: F1:0.6217469378906093, 	 P:0.7540050817115361, 	 R:0.6008991721044757

TEST: F1:0.6602743298977589, 	 P:0.6847569477913314, 	 R:0.6762607781875901
---------------------------------------------------------------
Working on:  roberta_bigru_2_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2906.58it/s]



DEV: F1:0.5624295552423878, 	 P:0.6851106606285635, 	 R:0.5284446561901651

TEST: F1:0.6447894607736772, 	 P:0.6866542233394399, 	 R:0.6348004281497527
---------------------------------------------------------------


In [37]:
dev.to_csv(path_to_drive+"trial.csv", index=False)

test.to_csv(path_to_drive+"test.csv", index=False)

utils.save_json(path_to_drive+"tsd_experiments_logs_roberta_models.json", roberta_models)

In [39]:
#RANDOM
random_models, dev, test = train_model("random", None, glove_embed_size,  
                        X_train_seq, y_train_seq, X_dev_seq, y_dev_seq, X_test_seq, y_test_seq, 
                        dev, X_dev, y_dev, dev_texts, test, X_test, y_test, test_texts, 
                        idx2tag, random_embedding=True)

Working on:  random_bilstm_1_layer


100%|██████████| 2000/2000 [00:00<00:00, 2900.86it/s]



DEV: F1:0.6013717564646831, 	 P:0.6634583577599503, 	 R:0.6700118666005247

TEST: F1:0.6155133257814981, 	 P:0.6008382240555085, 	 R:0.7177588162074627
---------------------------------------------------------------
Working on:  random_bilstm_1_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2832.13it/s]



DEV: F1:0.5811606453173995, 	 P:0.6332848679150577, 	 R:0.6660810047293568

TEST: F1:0.5880881217634297, 	 P:0.5704739406074326, 	 R:0.7150433123011998
---------------------------------------------------------------
Working on:  random_bigru_1_layer


...


DEV: F1:0.6063446196518567, 	 P:0.6705867101983531, 	 R:0.6727388738763372

TEST: F1:0.6043180318998752, 	 P:0.588736369523075, 	 R:0.7086446832790266
---------------------------------------------------------------
Working on:  random_bigru_2_layer


100%|██████████| 2000/2000 [00:00<00:00, 2905.53it/s]



DEV: F1:0.540634672429913, 	 P:0.605961730803107, 	 R:0.5955924486621861

TEST: F1:0.563312021216984, 	 P:0.564082812443873, 	 R:0.645032303970318
---------------------------------------------------------------
Working on:  random_bigru_2_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2861.75it/s]



DEV: F1:0.608753414696813, 	 P:0.680828434027895, 	 R:0.6441736616719723

TEST: F1:0.6263297001486408, 	 P:0.6209543204511783, 	 R:0.708628389078898
---------------------------------------------------------------


In [40]:
dev.to_csv(path_to_drive+"trial.csv", index=False)

test.to_csv(path_to_drive+"test.csv", index=False)

utils.save_json(path_to_drive+"tsd_experiments_logs_random_models.json", random_models)

In [41]:
#roberta_gpt2
roberta_gpt2_models, dev, test = train_model("roberta_gpt2", rotberta_gpt2_embeddings, rotberta_gpt2_embed_size, 
                        X_train_seq, y_train_seq, X_dev_seq, y_dev_seq, X_test_seq, y_test_seq, 
                        dev, X_dev, y_dev, dev_texts, test, X_test, y_test, test_texts, 
                        idx2tag, random_embedding=False)

Working on:  roberta_gpt2_bilstm_1_layer


100%|██████████| 2000/2000 [00:00<00:00, 2779.69it/s]



DEV: F1:0.631488781883551, 	 P:0.7472863570530754, 	 R:0.6255773280603533

TEST: F1:0.6637994215828757, 	 P:0.6710445507004218, 	 R:0.7004173664137382
---------------------------------------------------------------
Working on:  roberta_gpt2_bilstm_1_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2925.50it/s]



DEV: F1:0.5905574795274336, 	 P:0.6954844340211558, 	 R:0.575959276573947

TEST: F1:0.6636002377258563, 	 P:0.6766465871071105, 	 R:0.6876995062097072
---------------------------------------------------------------
Working on:  roberta_gpt2_bigru_1_layer


...


DEV: F1:0.6385462863193058, 	 P:0.6848430609804754, 	 R:0.7093600705270132

TEST: F1:0.6009698638637146, 	 P:0.5654262293226843, 	 R:0.7326543436139791
---------------------------------------------------------------
Working on:  roberta_gpt2_bigru_2_layer


100%|██████████| 2000/2000 [00:00<00:00, 2967.37it/s]



DEV: F1:0.6645191403512173, 	 P:0.7849396313006827, 	 R:0.6569312032532001

TEST: F1:0.6621193009798484, 	 P:0.6673837192718405, 	 R:0.7059530533971161
---------------------------------------------------------------
Working on:  roberta_gpt2_bigru_2_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2882.18it/s]


DEV: F1:0.521628977451789, 	 P:0.620969111403894, 	 R:0.49392825915085303

TEST: F1:0.6272428058629365, 	 P:0.6600237316454615, 	 R:0.6206681070864288
---------------------------------------------------------------


In [42]:
dev.to_csv(path_to_drive+"trial.csv", index=False)

test.to_csv(path_to_drive+"test.csv", index=False)

utils.save_json(path_to_drive+"tsd_experiments_logs_roberta_gpt2_models.json", 
                roberta_gpt2_models)

In [44]:
#glove_gpt2
glove_gp2_models, dev, test = train_model("glove_gpt2", glove_gpt2_embeddings, glove_transformer_embed_size, 
                        X_train_seq, y_train_seq, X_dev_seq, y_dev_seq, X_test_seq, y_test_seq, 
                        dev, X_dev, y_dev, dev_texts, test, X_test, y_test, test_texts, 
                        idx2tag, random_embedding=False)

Working on:  glove_gpt2_bilstm_1_layer


100%|██████████| 2000/2000 [00:00<00:00, 2985.75it/s]



DEV: F1:0.6184718936728081, 	 P:0.7408866948281574, 	 R:0.5978379883104998

TEST: F1:0.6696737556670149, 	 P:0.6910007993945712, 	 R:0.6888428321287376
---------------------------------------------------------------
Working on:  glove_gpt2_bilstm_1_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2874.21it/s]



DEV: F1:0.5838968879301514, 	 P:0.698979489364444, 	 R:0.5604632558230043

TEST: F1:0.6654267506408075, 	 P:0.6875329025622638, 	 R:0.677541515859361
---------------------------------------------------------------
Working on:  glove_gpt2_bigru_1_layer


...


DEV: F1:0.4452463473166817, 	 P:0.5198731468872133, 	 R:0.4265537222935052

TEST: F1:0.5598765168514392, 	 P:0.5812383706979296, 	 R:0.5560138237219746
---------------------------------------------------------------
Working on:  glove_gpt2_bigru_2_layer


100%|██████████| 2000/2000 [00:00<00:00, 2985.59it/s]



DEV: F1:0.6682348029467542, 	 P:0.7974511645083054, 	 R:0.6507761619943305

TEST: F1:0.6708107144545831, 	 P:0.6852103716515099, 	 R:0.7006069489623742
---------------------------------------------------------------
Working on:  glove_gpt2_bigru_2_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2814.88it/s]


DEV: F1:0.6015751387822573, 	 P:0.7395809267122699, 	 R:0.567556975399314

TEST: F1:0.6783873722305627, 	 P:0.7104342753200531, 	 R:0.6807853899511399
---------------------------------------------------------------


In [45]:
dev.to_csv(path_to_drive+"trial.csv", index=False)

test.to_csv(path_to_drive+"test.csv", index=False)

utils.save_json(path_to_drive+"tsd_experiments_logs_glove_gp2_models.json", 
                glove_gp2_models)

In [46]:
#glove_roberta
glove_roberta_models, dev, test = train_model("glove_roberta", glove_roberta_embeddings, glove_transformer_embed_size,
                        X_train_seq, y_train_seq, X_dev_seq, y_dev_seq, X_test_seq, y_test_seq, 
                        dev, X_dev, y_dev, dev_texts, test, X_test, y_test, test_texts, 
                        idx2tag, random_embedding=False)

Working on:  glove_roberta_bilstm_1_layer


100%|██████████| 2000/2000 [00:00<00:00, 3008.96it/s]



DEV: F1:0.6362389357257152, 	 P:0.747415567326646, 	 R:0.6247832793583947

TEST: F1:0.6793253621460283, 	 P:0.6890665947792657, 	 R:0.7130175487567275
---------------------------------------------------------------
Working on:  glove_roberta_bilstm_1_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2892.52it/s]



DEV: F1:0.6202944804947842, 	 P:0.733944462618241, 	 R:0.6010933407866489

TEST: F1:0.666222554021518, 	 P:0.6785412122615266, 	 R:0.6960622952630842
---------------------------------------------------------------
Working on:  glove_roberta_bigru_1_layer


...


DEV: F1:0.632944430057987, 	 P:0.6715569427250456, 	 R:0.7171007358094837

TEST: F1:0.6006739973544634, 	 P:0.5650225751465827, 	 R:0.7410764692891235
---------------------------------------------------------------
Working on:  glove_roberta_bigru_2_layer


100%|██████████| 2000/2000 [00:00<00:00, 2775.20it/s]



DEV: F1:0.6405921717881434, 	 P:0.7598120991683865, 	 R:0.6225781288644798

TEST: F1:0.6632798356207191, 	 P:0.6819519234647975, 	 R:0.6876261032825572
---------------------------------------------------------------
Working on:  glove_roberta_bigru_2_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2973.39it/s]


DEV: F1:0.06231884057971015, 	 P:0.06231884057971015, 	 R:0.06231884057971015

TEST: F1:0.197, 	 P:0.197, 	 R:0.197
---------------------------------------------------------------


In [47]:
dev.to_csv(path_to_drive+"trial.csv", index=False)

test.to_csv(path_to_drive+"test.csv", index=False)

utils.save_json(path_to_drive+"tsd_experiments_logs_glove_roberta_models.json", 
                glove_roberta_models)

In [48]:
#ensemble_10
ensemble_10_models, dev, test = train_model("ensemble_10", ensemble_embeddings, ensemble_embed_size, 
                        X_train_seq, y_train_seq, X_dev_seq, y_dev_seq, X_test_seq, y_test_seq, 
                        dev, X_dev, y_dev, dev_texts, test, X_test, y_test, test_texts, 
                        idx2tag, random_embedding=False, EPOCH=10)

Working on:  ensemble_10_bilstm_1_layer


100%|██████████| 2000/2000 [00:00<00:00, 2882.73it/s]



DEV: F1:0.6103977347735874, 	 P:0.7394568018915452, 	 R:0.5850066986557365

TEST: F1:0.6712893510888553, 	 P:0.6977682842764462, 	 R:0.6852374336927287
---------------------------------------------------------------
Working on:  ensemble_10_bilstm_1_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2877.21it/s]



DEV: F1:0.5834704756924469, 	 P:0.6850243768871781, 	 R:0.563494686858369

TEST: F1:0.6675900832583967, 	 P:0.6842633097501425, 	 R:0.6869184690709336
---------------------------------------------------------------
Working on:  ensemble_10_bigru_1_layer


...


DEV: F1:0.5358897559038032, 	 P:0.6469333001941696, 	 R:0.5068348772199864

TEST: F1:0.6460531747257343, 	 P:0.6799446821818532, 	 R:0.6394404326623537
---------------------------------------------------------------
Working on:  ensemble_10_bigru_2_layer


100%|██████████| 2000/2000 [00:00<00:00, 2904.20it/s]



DEV: F1:0.6435485066722026, 	 P:0.7782033486896186, 	 R:0.6199967840788086

TEST: F1:0.6807786611811747, 	 P:0.703421283913136, 	 R:0.6984812820127856
---------------------------------------------------------------
Working on:  ensemble_10_bigru_2_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2884.28it/s]


DEV: F1:0.6681107350075528, 	 P:0.7770345488416512, 	 R:0.6662403061236831

TEST: F1:0.6649351164190219, 	 P:0.6663665541692753, 	 R:0.713789784880434
---------------------------------------------------------------


In [49]:
dev.to_csv(path_to_drive+"trial.csv", index=False)

test.to_csv(path_to_drive+"test.csv", index=False)

utils.save_json(path_to_drive+"tsd_experiments_logs_ensemble_10_models.json", 
                ensemble_10_models)

In [50]:
#ensemble_20
ensemble_20_models, dev, test = train_model("ensemble_20", ensemble_embeddings, ensemble_embed_size, 
                        X_train_seq, y_train_seq, X_dev_seq, y_dev_seq, X_test_seq, y_test_seq, 
                        dev, X_dev, y_dev, dev_texts, test, X_test, y_test, test_texts, 
                        idx2tag, random_embedding=False, EPOCH=20)

Working on:  ensemble_20_bilstm_1_layer


100%|██████████| 2000/2000 [00:00<00:00, 2930.15it/s]



DEV: F1:0.6305000951066612, 	 P:0.7527759820913573, 	 R:0.6123807676081293

TEST: F1:0.6626822997127131, 	 P:0.6775839469029826, 	 R:0.6901171399677951
---------------------------------------------------------------
Working on:  ensemble_20_bilstm_1_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2906.30it/s]



DEV: F1:0.6585486731307924, 	 P:0.7511341001006716, 	 R:0.671652175706575

TEST: F1:0.6505622681080995, 	 P:0.638060842223474, 	 R:0.7255686154724711
---------------------------------------------------------------
Working on:  ensemble_20_bigru_1_layer


...


DEV: F1:0.6632448282464961, 	 P:0.7292925827548673, 	 R:0.701679687745869

TEST: F1:0.6335142111367086, 	 P:0.609457959096431, 	 R:0.7291280986138869
---------------------------------------------------------------
Working on:  ensemble_20_bigru_2_layer


100%|██████████| 2000/2000 [00:00<00:00, 2952.36it/s]



DEV: F1:0.589161771523194, 	 P:0.6113223945335668, 	 R:0.711010320584953

TEST: F1:0.5630028579709272, 	 P:0.5279794347184847, 	 R:0.7322080193719273
---------------------------------------------------------------
Working on:  ensemble_20_bigru_2_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2988.27it/s]


DEV: F1:0.6408746930523095, 	 P:0.7536154178399306, 	 R:0.633735947034072

TEST: F1:0.6552343729976636, 	 P:0.6694752406197387, 	 R:0.6928270615024656
---------------------------------------------------------------


In [51]:
dev.to_csv(path_to_drive+"trial.csv", index=False)

test.to_csv(path_to_drive+"test.csv", index=False)

utils.save_json(path_to_drive+"tsd_experiments_logs_ensemble_20_models.json", 
                ensemble_20_models)

In [54]:
#ensemble_15
ensemble_15_models_v2, dev, test = train_model("ensemble_15_v2", ensemble_embeddings, ensemble_embed_size, 
                        X_train_seq, y_train_seq, X_dev_seq, y_dev_seq, X_test_seq, y_test_seq, 
                        dev, X_dev, y_dev, dev_texts, test, X_test, y_test, test_texts, 
                        idx2tag, random_embedding=False, EPOCH=15)

Working on:  ensemble_15_v2_bilstm_1_layer


100%|██████████| 2000/2000 [00:00<00:00, 2986.90it/s]



DEV: F1:0.5969921148862494, 	 P:0.7091840065791136, 	 R:0.5758235204630973

TEST: F1:0.6633812955840966, 	 P:0.6872548671808928, 	 R:0.6772436036996925
---------------------------------------------------------------
Working on:  ensemble_15_v2_bilstm_1_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 3062.21it/s]



DEV: F1:0.623805817525997, 	 P:0.7347619959374206, 	 R:0.6125891856664127

TEST: F1:0.6620342940507341, 	 P:0.6727632854071592, 	 R:0.692799967131245
---------------------------------------------------------------
Working on:  ensemble_15_v2_bigru_1_layer


...


DEV: F1:0.5996224634256755, 	 P:0.7086970509689562, 	 R:0.5698404408475237

TEST: F1:0.6605324353581248, 	 P:0.6900665894430348, 	 R:0.6626661568334793
---------------------------------------------------------------
Working on:  ensemble_15_v2_bigru_2_layer


100%|██████████| 2000/2000 [00:00<00:00, 2903.65it/s]



DEV: F1:0.625367411452429, 	 P:0.7403570085857989, 	 R:0.612833720055414

TEST: F1:0.657054878516765, 	 P:0.6728541685958365, 	 R:0.6862945916586228
---------------------------------------------------------------
Working on:  ensemble_15_v2_bigru_2_layer_attention


100%|██████████| 2000/2000 [00:00<00:00, 2937.33it/s]


DEV: F1:0.6653563617890089, 	 P:0.7299043659723132, 	 R:0.7133133169357954

TEST: F1:0.6321012808185017, 	 P:0.6124051007438133, 	 R:0.7275604320824361
---------------------------------------------------------------


In [55]:
dev.to_csv(path_to_drive+"trial.csv", index=False)

test.to_csv(path_to_drive+"test.csv", index=False)

utils.save_json(path_to_drive+"tsd_experiments_logs_ensemble_15_models_v2.json", 
                ensemble_15_models_v2)

# 6. Store Results

In [ ]:
dev.to_csv(path_to_drive+"trial_experiments.csv", index=False)

test.to_csv(path_to_drive+"test_experiments.csv", index=False)

utils.save_json(path_to_drive+"tsd_experiments_logs.json", result)